### Import Required Modules

In [10]:
import pandas as pd
from pandas.io.json import json_normalize
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geometry import Geometry
from arcgis import GIS
import requests
import json
import time
import math
import datetime
import urllib.parse
import numpy as np

### Spire API Key

In [11]:
auth_token = "yzNObz76itGCSTOXEvx3JojZpfYFNrB8"

### AIS Query Parameters

In [12]:
API_endpoint = 'https://api.sense.spire.com/vessels'
output_format = 'json'
limit = 1000
waittime = 20 #in seconds
# Rotterdam
geoJSON = """{"type":"Polygon","coordinates":[[[3.9111328125000004,51.89513899196507],[4.211883544921875,51.89513899196507],[4.211883544921875,52.01531743663362],[3.9111328125000004,52.01531743663362],[3.9111328125000004,51.89513899196507]]]}"""
# Savannah
#geoJSON = """{"type":"Polygon","coordinates":[[[-81.31805419921875,31.836732676846065],[-80.76324462890625,31.836732676846065],[-80.76324462890625,32.29525895520317],[-81.31805419921875,32.29525895520317],[-81.31805419921875,31.836732676846065]]]}"""
# Philly
#geoJSON = """{"type":"Polygon","coordinates":[[[-75.21240234375,39.88036665897025],[-75.10030746459961,39.88036665897025],[-75.10030746459961,39.95541184288875],[-75.21240234375,39.95541184288875],[-75.21240234375,39.88036665897025]]]}"""

### Query AIS Data from SPIRE API

In [13]:
def queryAIS(now):
    #print('Start Querying SPIRE Data...')
    #now = datetime.datetime.now()
    newiso = datetime.datetime.isoformat(now)
    headers = {"Authorization": "Bearer {}".format(auth_token), 'Accept': 'application/{}'.format(output_format)}
    request = '{}/?updated_after={}&limit={}&last_known_position_within={}'.format(API_endpoint,newiso,limit,geoJSON) 
    response = requests.get(request, headers=headers)
    data = response.json()
    datajson = json.loads(response.text)
    return datajson
    #print('Done.')

### Create a Spatially Enabled DataFrame Using the ArcGIS API for Python

In [14]:
def getRotatedCoords(x,y,xc,yc,angle):
    x = x - xc  
    y = y - yc
    angle = angle * -1  
    angle = math.radians(angle)  
    xr = (x * math.cos(angle)) - (y * math.sin(angle)) + xc  
    yr = (x * math.sin(angle)) + (y * math.cos(angle)) + yc  
    return [xr, yr]   

In [15]:
def point_to_poly(row):
    #get data from the Series
    coords = row["last_known_position.geometry.coordinates"]
    rotation = row["last_known_position.heading"]
    if rotation > 360:
        rotation = 0
    if row["length"] > 0:
        shipLength = row["length"]
    else:
        shipLength = 10
    if row["width"] > 0:
        shipWidth = row["width"]
    else:
        shipWidth = 4
    x_lon = coords[0]
    y_lat = coords[1]
    
    #Convert GCS to Web Mercator Coordinates
    if abs(x_lon) <= 180 and abs(y_lat) < 90:          
        num = x_lon * 0.017453292519943295         
        x = 6378137.0 * num         
        a = y_lat * 0.017453292519943295          
        x_mercator = x         
        y_mercator = 3189068.5 * math.log((1.0 + math.sin(a)) / (1.0 - math.sin(a)))

        polygon = [
            [x_mercator - (shipWidth * 0.5), y_mercator - (shipLength * 0.5)],
            [x_mercator - (shipWidth * 0.5), y_mercator + (shipLength * 0.350)],
            [x_mercator - (shipWidth * 0.431), y_mercator + (shipLength * 0.392)],
            [x_mercator - (shipWidth * 0.357), y_mercator + (shipLength * 0.427)],            
            [x_mercator - (shipWidth * 0.281), y_mercator + (shipLength * 0.455)],
            [x_mercator - (shipWidth * 0.203), y_mercator + (shipLength * 0.478)],
            [x_mercator - (shipWidth * 0.123), y_mercator + (shipLength * 0.492)],     
            [x_mercator - (shipWidth * 0.088), y_mercator + (shipLength * 0.495)],
            [x_mercator,y_mercator + (shipLength * 0.5)],
            [x_mercator,y_mercator + (shipLength * 0.5)],
            [x_mercator + (shipWidth * 0.088), y_mercator + (shipLength * 0.495)],
            [x_mercator + (shipWidth * 0.123), y_mercator + (shipLength * 0.492)],
            [x_mercator + (shipWidth * 0.203), y_mercator + (shipLength * 0.478)],
            [x_mercator + (shipWidth * 0.281), y_mercator + (shipLength * 0.455)],
            [x_mercator + (shipWidth * 0.357), y_mercator + (shipLength * 0.427)],
            [x_mercator + (shipWidth * 0.431), y_mercator + (shipLength * 0.392)],
            [x_mercator + (shipWidth * 0.5), y_mercator + (shipLength * 0.350)],                                                
            [x_mercator + (shipWidth * 0.5), y_mercator - (shipLength * 0.5)],
            [x_mercator - (shipWidth * 0.5), y_mercator - (shipLength * 0.5)],
        ]        
        
        #Get Rotated Coordinates
        rotatedPoly = []
        for vertex in polygon:
            rotatedPoly.append(getRotatedCoords(vertex[0],vertex[1],x_mercator,y_mercator,rotation))  
        
        return Geometry({"rings": [rotatedPoly],
                "spatialReference": {"wkid": 102100}})
    else:
        return None

In [16]:
def createSpatialDataFrame(datajson):
    df = pd.DataFrame(data=json_normalize(datajson['data']))
    df['SHAPE']=df.apply(point_to_poly, axis=1)
    df.spatial.set_geometry('SHAPE')
    return df

### Create the Map

In [17]:
aisMap = GIS().map('Europoort')
aisMap.basemap = 'satellite'
aisMap

MapView(layout=Layout(height='400px', width='100%'))

In [18]:
for i in range(1):
    shipLocations = queryAIS(datetime.datetime.now())
    df = createSpatialDataFrame(shipLocations)
    df.spatial.plot(map_widget=aisMap,
                           cmap = [255,255,255,200],
                           symbol_type='simple',
                           symbol_style='s',
                           outline_style='s',
                           outline_color=[0,0,0,255],
                           line_width=0.5)
        
    print('Total AIS responses: {}'.format(len(df)))
    time.sleep(waittime)

/Users/josh6643/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
/Users/josh6643/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


Total AIS responses: 265


In [20]:
df

,ais_version,call_sign,class,created_at,flag,general_classification,gross_tonnage,id,imo,individual_classification,...,predicted_position.geometry.coordinates,predicted_position.geometry.type,predicted_position.speed,predicted_position.timestamp,predictions,ship_type,static_updated_at,updated_at,width,SHAPE
0,0,PHGI,A,2017-08-15T19:45:05.691886+00:00,NL,None,None,c02210e7-8c78-4495-a372-488f9d2aee23,9373888.0,None,...,[],Point,NaN,None,[],Other,2019-05-28T07:39:33.735085+00:00,2019-05-28T14:25:12.407363+00:00,6.0,"{""rings"": [[[459780.3308946855, 6791573.956077..."
1,0,A8WY7,A,2017-09-13T16:11:22.409073+00:00,LR,Merchant,0,67b94be3-824e-424f-8b63-d57d4e025342,9472086.0,Bulk Carrier,...,[],Point,NaN,None,[],Cargo,2019-05-10T11:50:41.468529+00:00,2019-05-28T14:17:08.763630+00:00,32.0,"{""rings"": [[[459522.5278810864, 6791794.000913..."
2,0,P3CJ9,A,2017-09-17T21:56:38.974060+00:00,CY,None,None,73bd8fb0-73c6-4642-ac51-200044d5dfd5,9234991.0,None,...,[],Point,NaN,None,[],Cargo,2019-05-23T10:49:06.215213+00:00,2019-05-28T14:22:11.874073+00:00,22.0,"{""rings"": [[[449691.58312915196, 6795225.82765..."
3,0,PE5951,A,2017-09-27T04:29:30.893427+00:00,NL,None,None,ef352f19-96c2-4671-babc-716382385641,NaN,None,...,[],Point,NaN,None,[],Cargo,2019-03-19T18:03:11.297221+00:00,2019-05-28T14:23:10.578859+00:00,5.0,"{""rings"": [[[466278.11150106124, 6789506.08103..."
4,0,PDKT,B,2017-09-27T04:29:31.943314+00:00,NL,None,None,230a3b11-464f-4610-8de9-263506ed4674,9820623.0,None,...,[],Point,NaN,None,[],Tug,2019-03-15T15:32:33.991999+00:00,2019-05-28T14:21:15.127252+00:00,13.0,"{""rings"": [[[460891.274169629, 6793113.8890262..."
5,0,2EPG9,A,2017-10-11T06:13:34.894207+00:00,GB,Merchant,18,7f22413c-02ad-4c71-a7a3-d57cd3b8f83d,NaN,Patrol Ship,...,[],Point,NaN,None,[],Other,2019-05-28T10:58:36.997979+00:00,2019-05-28T14:21:15.127252+00:00,5.0,"{""rings"": [[[465439.87573538785, 6790524.28799..."
6,0,PA4609,A,2017-11-02T11:57:34.827707+00:00,NL,None,None,a20e0747-01fa-42fb-873d-b2d673a2df48,NaN,None,...,[],Point,NaN,None,[],Cargo,2019-04-24T04:45:35.324919+00:00,2019-05-28T14:21:13.593404+00:00,7.0,"{""rings"": [[[458680.6694493282, 6791606.933779..."
7,0,PC4462,B,2017-11-12T09:46:32.313970+00:00,NL,None,None,e5e94b7b-7658-41a4-8b3b-6a0c3ed30457,NaN,None,...,"[4.04141, 51.94664]",Point,0.0,2019-05-28T14:02:41+00:00,[],Sailing Vessel,2019-05-02T08:31:46.581271+00:00,2019-05-28T11:03:10.467623+00:00,3.0,"{""rings"": [[[449886.2032868438, 6790478.026484..."
8,0,PI2976,A,2017-11-13T05:47:33.151565+00:00,NL,None,None,fd6bf5c5-351f-46a4-a392-b5c51fe2702f,NaN,None,...,[],Point,NaN,None,[],Cargo,2019-05-01T00:16:47.489794+00:00,2019-05-28T14:22:12.726472+00:00,9.0,"{""rings"": [[[461036.30306942185, 6793683.59613..."
9,0,PI9728,A,2017-11-17T07:13:26.866278+00:00,NL,None,None,6be55920-7df7-48db-a734-a79167e55e30,NaN,None,...,"[4.13795, 51.9613]",Point,0.0,2019-05-28T14:05:22+00:00,[],Other,2019-03-20T06:51:17.938932+00:00,2019-05-28T13:06:22.470082+00:00,3.0,"{""rings"": [[[460632.98692802637, 6793126.02052..."
